In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./kit.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_info = df['Kit_Info'].tolist()

print('总数量：' + str(len(list_info)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for info in list_info:
    work.put_nowait(info)

def crawler():
    global df
    
    while not work.empty():
        info = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(info,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_oe = []
                oe = html.xpath('//section[contains(@aria-label, "Alternate/OEM Part Number(s)")]/text()')
                for each_oe in oe:
                    list_each_oe = each_oe.split(',')
                    for _ in list_each_oe:
                        list_oe.append(_)

                oe = ';'.join([_.replace('}', '').replace('OEM / Interchange Numbers:', '').strip() for _ in list_oe])
                
                # = = = = = = = = = = = = = = =       
                
                src = html.xpath('//img[contains(@alt, "Part image")]/@src')
                if len(src) == 0:
                    src = ''
                else:
                    src = 'https://www.rockauto.com' + src[0].strip()
                    
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Kit_Info': info,
                                         'Kit_Alternate_OE_Part_Numbers': oe,
                                         'Kit_Src': src}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Kit_Info': info}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(info +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./kit_specification_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：1

https://www.rockauto.com/en/moreinfo.php?pt=5728&pk=8307440  <->  [ok] - 剩余数量：0

爬虫输出中

爬虫结束
